# 前述
DCGAN就是Deep Concolutions應用到GAN上，但是和傳統的卷積應用還有一些區別，最大的區別就是沒有池化層。

# 具體
## DCGAN和傳統GAN區別
### 將pooling層convolutions替代
對於判別模型：容許網絡學習自己的空間下採樣 ，因為沒有池化層，所以讓判別網絡自習學習。對於生成模型：容許它學習自己的空間上採樣，即改變原先卷積策略，通過圖片提取特徵，而生成模型通過特徵生成圖片。）

### 在generator和discriminator上都使用batchnorm
解決初始化差的問題
幫助梯度傳播到每一層
防止generator把所有的樣本都收斂到同一個點。

### 在CNN中移除全連接層

### 在generator的除了輸出層外的所有層使用ReLU，輸出層採用tanh。

### 在discriminator的所有層上使用LeakyReLU

In [1]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys

import numpy as np

class DCGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    def build_generator(self):

        model = Sequential()

        model.add(Dense(128 * 7 * 7, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((7, 7, 128)))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(self.channels, kernel_size=3, padding="same"))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))

        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, save_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = X_train / 127.5 - 1.
        X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            # Sample noise and generate a batch of new images
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator (real classified as ones and generated as zeros)
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            # Train the generator (wants discriminator to mistake images as real)
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f, acc.: %.2f%%]" % (epoch, d_loss[0], 100*d_loss[1], g_loss[0], g_loss[1]))

            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                self.save_imgs(epoch)

    def save_imgs(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/mnist_%d.png" % epoch)
        plt.close()


if __name__ == '__main__':
    dcgan = DCGAN()
    dcgan.train(epochs=4000, batch_size=32, save_interval=50)


Using TensorFlow backend.
W0216 14:08:13.518400 4429972928 deprecation.py:506] From /Users/ianfan/anaconda3/envs/spinningup/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0216 14:08:13.693822 4429972928 deprecation.py:506] From /Users/ianfan/anaconda3/envs/spinningup/lib/python3.6/site-packages/tensorflow/python/training/moving_averages.py:210: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 14, 14, 32)        320       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)          18496     
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 8, 8, 64)          0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 8, 8, 64)          256       
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 8, 8, 64)          0         
__________

/Users/ianfan/anaconda3/envs/spinningup/lib/python3.6/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 1.149144, acc.: 32.81%] [G loss: 0.862038, acc.: 0.56%]
1 [D loss: 0.582537, acc.: 64.06%] [G loss: 1.236757, acc.: 0.19%]
2 [D loss: 0.505921, acc.: 73.44%] [G loss: 1.256343, acc.: 0.19%]
3 [D loss: 0.286628, acc.: 93.75%] [G loss: 1.284161, acc.: 0.31%]
4 [D loss: 0.229126, acc.: 95.31%] [G loss: 0.893322, acc.: 0.44%]
5 [D loss: 0.218636, acc.: 93.75%] [G loss: 0.813202, acc.: 0.62%]
6 [D loss: 0.190613, acc.: 98.44%] [G loss: 0.892951, acc.: 0.56%]
7 [D loss: 0.319930, acc.: 89.06%] [G loss: 1.034045, acc.: 0.31%]
8 [D loss: 0.656366, acc.: 65.62%] [G loss: 1.473039, acc.: 0.22%]
9 [D loss: 0.723997, acc.: 64.06%] [G loss: 2.474424, acc.: 0.00%]
10 [D loss: 1.223228, acc.: 28.12%] [G loss: 2.283019, acc.: 0.00%]
11 [D loss: 0.830917, acc.: 62.50%] [G loss: 1.871459, acc.: 0.06%]
12 [D loss: 0.528374, acc.: 68.75%] [G loss: 1.348374, acc.: 0.31%]
13 [D loss: 0.819076, acc.: 57.81%] [G loss: 1.406750, acc.: 0.22%]
14 [D loss: 0.644955, acc.: 68.75%] [G loss: 1.459386, acc

121 [D loss: 0.870920, acc.: 46.88%] [G loss: 1.320154, acc.: 0.19%]
122 [D loss: 0.741495, acc.: 54.69%] [G loss: 1.206486, acc.: 0.25%]
123 [D loss: 0.854732, acc.: 56.25%] [G loss: 1.377656, acc.: 0.06%]
124 [D loss: 1.053974, acc.: 37.50%] [G loss: 1.361998, acc.: 0.28%]
125 [D loss: 0.941276, acc.: 43.75%] [G loss: 1.313627, acc.: 0.25%]
126 [D loss: 0.747000, acc.: 56.25%] [G loss: 1.116737, acc.: 0.28%]
127 [D loss: 0.982560, acc.: 43.75%] [G loss: 1.291853, acc.: 0.12%]
128 [D loss: 0.892696, acc.: 46.88%] [G loss: 1.335753, acc.: 0.28%]
129 [D loss: 0.809876, acc.: 64.06%] [G loss: 1.212456, acc.: 0.25%]
130 [D loss: 1.052566, acc.: 35.94%] [G loss: 1.188458, acc.: 0.28%]
131 [D loss: 0.851006, acc.: 53.12%] [G loss: 1.032823, acc.: 0.22%]
132 [D loss: 0.921243, acc.: 46.88%] [G loss: 1.001746, acc.: 0.41%]
133 [D loss: 0.848325, acc.: 42.19%] [G loss: 1.202181, acc.: 0.19%]
134 [D loss: 0.762006, acc.: 51.56%] [G loss: 1.429892, acc.: 0.12%]
135 [D loss: 0.796789, acc.: 54.69

...

3432 [D loss: 0.596730, acc.: 70.31%] [G loss: 1.114862, acc.: 0.22%]
3433 [D loss: 0.670923, acc.: 56.25%] [G loss: 0.967304, acc.: 0.25%]
3434 [D loss: 0.607136, acc.: 67.19%] [G loss: 1.209135, acc.: 0.19%]
3435 [D loss: 0.587246, acc.: 65.62%] [G loss: 1.062071, acc.: 0.34%]
3436 [D loss: 0.598515, acc.: 70.31%] [G loss: 1.072483, acc.: 0.16%]
3437 [D loss: 0.659667, acc.: 57.81%] [G loss: 0.821206, acc.: 0.38%]
3438 [D loss: 0.689631, acc.: 59.38%] [G loss: 0.957277, acc.: 0.25%]
3439 [D loss: 0.616201, acc.: 59.38%] [G loss: 0.950156, acc.: 0.28%]
3440 [D loss: 0.650057, acc.: 62.50%] [G loss: 1.091472, acc.: 0.19%]
3441 [D loss: 0.653593, acc.: 62.50%] [G loss: 0.991076, acc.: 0.28%]
3442 [D loss: 0.719792, acc.: 50.00%] [G loss: 0.843131, acc.: 0.31%]
3443 [D loss: 0.694058, acc.: 54.69%] [G loss: 0.890893, acc.: 0.22%]
3444 [D loss: 0.552042, acc.: 71.88%] [G loss: 0.953486, acc.: 0.28%]
3445 [D loss: 0.635928, acc.: 60.94%] [G loss: 1.097474, acc.: 0.16%]
3446 [D loss: 0.6873

3550 [D loss: 0.674653, acc.: 57.81%] [G loss: 1.384318, acc.: 0.06%]
3551 [D loss: 0.610645, acc.: 64.06%] [G loss: 0.958200, acc.: 0.31%]
3552 [D loss: 0.616028, acc.: 68.75%] [G loss: 1.004909, acc.: 0.19%]
3553 [D loss: 0.641642, acc.: 65.62%] [G loss: 1.007290, acc.: 0.12%]
3554 [D loss: 0.564995, acc.: 75.00%] [G loss: 1.046602, acc.: 0.25%]
3555 [D loss: 0.629590, acc.: 65.62%] [G loss: 0.986964, acc.: 0.19%]
3556 [D loss: 0.712078, acc.: 57.81%] [G loss: 0.933284, acc.: 0.31%]
3557 [D loss: 0.597477, acc.: 70.31%] [G loss: 1.109770, acc.: 0.16%]
3558 [D loss: 0.685600, acc.: 54.69%] [G loss: 0.964177, acc.: 0.31%]
3559 [D loss: 0.575744, acc.: 68.75%] [G loss: 1.079153, acc.: 0.19%]
3560 [D loss: 0.731556, acc.: 57.81%] [G loss: 0.975806, acc.: 0.28%]
3561 [D loss: 0.600462, acc.: 73.44%] [G loss: 1.126072, acc.: 0.19%]
3562 [D loss: 0.591345, acc.: 62.50%] [G loss: 1.082558, acc.: 0.22%]
3563 [D loss: 0.684686, acc.: 57.81%] [G loss: 0.978362, acc.: 0.25%]
3564 [D loss: 0.6021

3668 [D loss: 0.620415, acc.: 65.62%] [G loss: 0.874392, acc.: 0.28%]
3669 [D loss: 0.699027, acc.: 56.25%] [G loss: 0.876934, acc.: 0.31%]
3670 [D loss: 0.664421, acc.: 54.69%] [G loss: 0.981804, acc.: 0.19%]
3671 [D loss: 0.630854, acc.: 59.38%] [G loss: 1.157496, acc.: 0.09%]
3672 [D loss: 0.688884, acc.: 56.25%] [G loss: 0.825010, acc.: 0.50%]
3673 [D loss: 0.633199, acc.: 59.38%] [G loss: 1.144932, acc.: 0.28%]
3674 [D loss: 0.685313, acc.: 62.50%] [G loss: 0.932327, acc.: 0.31%]
3675 [D loss: 0.718105, acc.: 50.00%] [G loss: 1.084838, acc.: 0.12%]
3676 [D loss: 0.733997, acc.: 50.00%] [G loss: 1.019103, acc.: 0.31%]
3677 [D loss: 0.629197, acc.: 64.06%] [G loss: 0.904469, acc.: 0.31%]
3678 [D loss: 0.617112, acc.: 71.88%] [G loss: 1.010658, acc.: 0.25%]
3679 [D loss: 0.753994, acc.: 51.56%] [G loss: 0.927707, acc.: 0.38%]
3680 [D loss: 0.770539, acc.: 46.88%] [G loss: 1.004375, acc.: 0.22%]
3681 [D loss: 0.600847, acc.: 68.75%] [G loss: 1.037603, acc.: 0.19%]
3682 [D loss: 0.5551

3786 [D loss: 0.647079, acc.: 60.94%] [G loss: 0.989857, acc.: 0.31%]
3787 [D loss: 0.759036, acc.: 45.31%] [G loss: 0.819347, acc.: 0.38%]
3788 [D loss: 0.679408, acc.: 64.06%] [G loss: 0.917055, acc.: 0.34%]
3789 [D loss: 0.663092, acc.: 57.81%] [G loss: 1.129693, acc.: 0.19%]
3790 [D loss: 0.645931, acc.: 56.25%] [G loss: 0.931121, acc.: 0.28%]
3791 [D loss: 0.682552, acc.: 60.94%] [G loss: 0.953229, acc.: 0.34%]
3792 [D loss: 0.585555, acc.: 71.88%] [G loss: 1.076291, acc.: 0.09%]
3793 [D loss: 0.656367, acc.: 70.31%] [G loss: 0.944482, acc.: 0.31%]
3794 [D loss: 0.692466, acc.: 60.94%] [G loss: 1.035728, acc.: 0.19%]
3795 [D loss: 0.553893, acc.: 70.31%] [G loss: 1.193429, acc.: 0.06%]
3796 [D loss: 0.574020, acc.: 76.56%] [G loss: 1.082560, acc.: 0.12%]
3797 [D loss: 0.695415, acc.: 56.25%] [G loss: 1.132595, acc.: 0.16%]
3798 [D loss: 0.639331, acc.: 60.94%] [G loss: 0.923571, acc.: 0.25%]
3799 [D loss: 0.677129, acc.: 57.81%] [G loss: 0.853840, acc.: 0.44%]
3800 [D loss: 0.6123

3904 [D loss: 0.760786, acc.: 46.88%] [G loss: 1.108896, acc.: 0.12%]
3905 [D loss: 0.650435, acc.: 64.06%] [G loss: 0.931761, acc.: 0.38%]
3906 [D loss: 0.706286, acc.: 57.81%] [G loss: 0.945586, acc.: 0.31%]
3907 [D loss: 0.683098, acc.: 60.94%] [G loss: 0.930086, acc.: 0.28%]
3908 [D loss: 0.614305, acc.: 67.19%] [G loss: 1.058993, acc.: 0.25%]
3909 [D loss: 0.612244, acc.: 67.19%] [G loss: 1.187517, acc.: 0.22%]
3910 [D loss: 0.681976, acc.: 48.44%] [G loss: 0.917546, acc.: 0.28%]
3911 [D loss: 0.681606, acc.: 62.50%] [G loss: 1.069646, acc.: 0.16%]
3912 [D loss: 0.692897, acc.: 60.94%] [G loss: 0.861108, acc.: 0.31%]
3913 [D loss: 0.655668, acc.: 64.06%] [G loss: 1.041051, acc.: 0.28%]
3914 [D loss: 0.580573, acc.: 68.75%] [G loss: 1.002311, acc.: 0.31%]
3915 [D loss: 0.664975, acc.: 60.94%] [G loss: 1.065971, acc.: 0.16%]
3916 [D loss: 0.651517, acc.: 65.62%] [G loss: 0.923315, acc.: 0.41%]
3917 [D loss: 0.707900, acc.: 54.69%] [G loss: 0.922366, acc.: 0.38%]
3918 [D loss: 0.6402

In [ ]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys

import numpy as np

In [ ]:
class DCGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    def build_generator(self):

        model = Sequential()

        model.add(Dense(128 * 7 * 7, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((7, 7, 128)))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(self.channels, kernel_size=3, padding="same"))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))

        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, save_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = X_train / 127.5 - 1.
        X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            # Sample noise and generate a batch of new images
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator (real classified as ones and generated as zeros)
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            # Train the generator (wants discriminator to mistake images as real)
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f, acc.: %.2f%%]" % (epoch, d_loss[0], 100*d_loss[1], g_loss[0], g_loss[1]))

            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                self.save_imgs(epoch)

    def save_imgs(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/mnist_%d.png" % epoch)
        plt.close()

In [ ]:
if __name__ == '__main__':
    dcgan = DCGAN()
    dcgan.train(epochs=4000, batch_size=32, save_interval=100)